In [123]:
import numpy as np
import pandas as pd
from dotenv import load_dotenv
import os
import alpaca_trade_api as tradeapi
from trade import trade_action
from datetime import datetime, timedelta

In [124]:
def market_data( start, end, tickers ):
    load_dotenv()
    alpaca_api_key = os.getenv('ALPACA_API_KEY')
    alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')

    # Create the Alpaca API object
    alpaca_api = tradeapi.REST(
       alpaca_api_key,
       alpaca_secret_key,
       api_version = 'v2'
    )
    
    # Set timeframe to "1Day" for Alpaca API
    timeframe = "1Day"
    
    start_date = pd.Timestamp(start,tz='America/New_York')
    end_date = pd.Timestamp(end,tz='America/New_York')
           
    # Get number_of_years' worth of historical data for tickers
    data_df = alpaca_api.get_bars(
        tickers,
        timeframe,
        start = start_date.isoformat(),
        end = end_date.isoformat()
    ).df
    if len(data_df) == 0:
        return []
    return data_df

#df_market = market_data( '2023-08-01', '2023-08-21', tickers=['TSLA'] )

## Plan
* Get all ticker data for a year
* Get one ticker, get signal column
* Slice backwards, 15 for pred, 30 for train
* Scale train
* Rolling train, 30 at a time, predict one day at a time, till the end
* Store the prediction in my instr_df

In [154]:
today = datetime.today().date()
tickers = ['AAPL', 'AMZN', 'MSFT', 'GOOG']
date_from = today - timedelta(days=365)
data_df = market_data(date_from, today, tickers)
aapl_df = data_df[data_df['symbol']=='AAPL']
aapl_cycle = aapl_df.iloc[-45:,:]
#aapl_cycle
line_up = pd.concat([aapl_cycle.drop('symbol',axis=1), aapl_cycle.drop('symbol',axis=1).shift(1)],
                    axis=1, 
                    join='outer')
line_up.columns = list(aapl_cycle.drop('symbol',axis=1).columns.values) + ['openl','highl','lowl','closel','volumel','tradecountl','vwapl']
line_up

,open,high,low,close,volume,trade_count,vwap,openl,highl,lowl,closel,volumel,tradecountl,vwapl
timestamp,,,,,,,,,,,,,,
2023-06-27 04:00:00+00:00,185.8900,188.3900,185.6700,188.060,50781649,513088,187.500148,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-06-28 04:00:00+00:00,187.9300,189.9000,187.6000,189.250,51239329,560503,188.892040,185.8900,188.3900,185.6700,188.060,50781649.0,513088.0,187.500148
2023-06-29 04:00:00+00:00,189.0800,190.0700,188.9400,189.590,46347308,525367,189.556085,187.9300,189.9000,187.6000,189.250,51239329.0,560503.0,188.892040
2023-06-30 04:00:00+00:00,191.7750,194.4800,191.2600,193.880,68445760,758835,192.898344,189.0800,190.0700,188.9400,189.590,46347308.0,525367.0,189.556085
2023-07-03 04:00:00+00:00,193.7800,193.8800,191.7600,192.460,31458198,500175,192.662368,191.7750,194.4800,191.2600,193.880,68445760.0,758835.0,192.898344
2023-07-05 04:00:00+00:00,191.5650,192.9800,190.6200,191.330,46961338,561201,191.580869,193.7800,193.8800,191.7600,192.460,31458198.0,500175.0,192.662368
2023-07-06 04:00:00+00:00,189.9000,192.0200,189.2000,191.810,45156467,562757,190.821416,191.5650,192.9800,190.6200,191.330,46961338.0,561201.0,191.580869
2023-07-07 04:00:00+00:00,191.4100,192.6700,190.2400,190.680,46676638,536324,191.419203,189.9000,192.0200,189.2000,191.810,45156467.0,562757.0,190.821416
2023-07-10 04:00:00+00:00,189.2600,189.9900,187.0350,188.610,59931763,736913,188.363044,191.4100,192.6700,190.2400,190.680,46676638.0,536324.0,191.419203


In [ ]:
# get signals - daily return method
aapl_signal = aapl_cycle.loc[:,['vwap']]aapl_signal['daily_return'] = aapl_signal['vwap'].pct_change()
aapl_signal['lagged_daily_return'] = aapl_signal['daily_return'].shift(1)
aapl_signal['twice_lagged'] = aapl_signal['daily_return'].shift(2)

In [132]:
aapl_signal[['daily_return','lagged_daily_return','twice_lagged']].hvplot(frame_width=700)

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

In [ ]:
# aapl_signal['signal'] = np.where((aapl_signal['lagged_daily_return']>0) &
#                                  (aapl_signal['twice_lagged']>0), -1, 
#                                  np.where((aapl_signal['lagged_daily_return']<0) &
#                                           (aapl_signal['twice_lagged']<0), 1, 0 
#                                          )
#                                 )
# aapl_signal

In [42]:
X = aapl_cycle.drop('symbol',axis=1)
y = aapl_signal['signal']

## scaler method

In [43]:
from sklearn.preprocessing import StandardScaler
stsc = StandardScaler()
stsc.fit(X)
X_scaled = stsc.transform(X)

In [134]:
X_scaled 
    

array([[-1.41665645e-02, -2.83536788e-02, -2.60953534e-02,
        -1.93254342e-01, -3.58962365e-01, -1.54695080e-01,
        -5.92688225e-02],
       [-1.48019382e-01,  1.95532139e-02,  3.44727122e-02,
         1.95483664e-01, -1.87879691e-01, -6.12534472e-01,
         1.04372121e-01],
       [ 1.42469712e-01,  2.32316179e-01,  3.00146273e-01,
         3.61289480e-01, -1.57848979e-01, -3.13918920e-01,
         2.98589460e-01],
       [ 3.06225818e-01,  2.56269625e-01,  4.84603564e-01,
         4.08662570e-01, -4.78839354e-01, -5.35202396e-01,
         3.91246830e-01],
       [ 6.89984695e-01,  8.77650204e-01,  8.03962455e-01,
         1.00639950e+00,  9.71152460e-01,  9.35158787e-01,
         8.57608191e-01],
       [ 9.75489907e-01,  7.93108629e-01,  8.72789803e-01,
         8.08547186e-01, -1.45578958e+00, -6.93859424e-01,
         8.24681333e-01],
       [ 6.60081406e-01,  6.66296266e-01,  7.15863451e-01,
         6.51101327e-01, -4.38549723e-01, -3.09522977e-01,
         6.7377461

In [3]:
# build a random instruction dataframe example:
#df = pd.DataFrame([['2023-08-24', 'TSLA', 1], 
#                   ['2023-08-25', 'AAPL', 1])
tickers = ['TSLA','AAPL','MSFT','AMZN']
instrs = np.random.randn(50, 1)

def sample_df():
    instri = 0
    start_date = pd.to_datetime('2023-08-01').normalize()

    # using alpaca data Aug 01 - Aug 21 for the dates to avoid non-trading days
    df = pd.DataFrame()
    for index, data in df_market.iterrows():
        if instrs[instri] > 0:
            instr = 1
        else:
            instr = -1
        instri += 1
        one_row = pd.DataFrame([[index.date(), tickers[np.random.randint(4)], instr]])
        df = pd.concat([df,one_row], join='outer')
    return df
    

In [8]:
final_worths = []
for n in range(500):
    final_worths.append(trade_action( sample_df(), verbose=0 ))

In [9]:
pd.DataFrame(final_worths).hvplot(kind='hist')

:Histogram   [0]   (0_count)

In [10]:
trade_action(sample_df())

On 2023-08-01 trade TSLA
{'action': 'buy', 'price': 263.51, 'bal': 7628.41, 'share': 9, 'status': 0, 'msg': 'success'}
On 2023-08-02 trade TSLA
{'action': 'sell', 'price': 254.92, 'bal': 9922.69, 'share': 9, 'status': 0, 'msg': 'success'}
On 2023-08-03 trade AMZN
{'action': 'none', 'price': 128.16, 'bal': 9922.69, 'share': 0, 'status': -1, 'msg': 'no share to sell'}
On 2023-08-04 trade MSFT
{'action': 'buy', 'price': 330.51, 'bal': 7609.12, 'share': 7, 'status': 0, 'msg': 'success'}
On 2023-08-07 trade MSFT
{'action': 'sell', 'price': 329.28, 'bal': 9914.08, 'share': 7, 'status': 0, 'msg': 'success'}
On 2023-08-08 trade TSLA
{'action': 'buy', 'price': 248.27, 'bal': 7431.38, 'share': 10, 'status': 0, 'msg': 'success'}
On 2023-08-09 trade AMZN
{'action': 'buy', 'price': 138.81, 'bal': 4932.8, 'share': 18, 'status': 0, 'msg': 'success'}
On 2023-08-10 trade AMZN
{'action': 'sell', 'price': 138.88, 'bal': 7432.64, 'share': 18, 'status': 0, 'msg': 'success'}
On 2023-08-11 trade MSFT
{'actio

:Layout
   .NdOverlay.I  :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .NdOverlay.II :NdOverlay   [Variable]
      :Curve   [date]   (value)

# Kmeans cluster
## pct_change method

In [59]:
aapl_unsup = ( aapl_df.drop('symbol',axis=1).copy() ).pct_change().dropna()
aapl_unsup

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2022-08-30 04:00:00+00:00,0.006113,-0.002087,-0.013140,-0.015305,0.062506,0.006230,-0.013845
2022-08-31 04:00:00+00:00,-0.011256,-0.012180,-0.003677,-0.010635,0.128996,-0.059123,-0.005622
2022-09-01 04:00:00+00:00,-0.022863,-0.013451,-0.015718,0.004707,-0.156010,0.079367,-0.011830
2022-09-02 04:00:00+00:00,0.019854,0.012259,0.001907,-0.013611,0.038395,-0.012934,0.007209
2022-09-06 04:00:00+00:00,-0.020532,-0.020404,-0.008228,-0.008215,-0.044042,0.063443,-0.016744
...,...,...,...,...,...,...,...
2023-08-23 04:00:00+00:00,0.008246,0.021781,0.011773,0.021949,0.253648,0.172417,0.020559
2023-08-24 04:00:00+00:00,0.012066,-0.002457,-0.012982,-0.026170,0.040973,0.036305,-0.016302
2023-08-25 04:00:00+00:00,-0.018232,-0.010789,-0.001079,0.012643,-0.074361,-0.103186,0.001167


In [61]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3, n_init=99)
km.fit(aapl_unsup)
km.inertia_

9.467484969601138

In [62]:
clusters = km.predict(aapl_unsup)
clusters

array([0, 0, 1, 0, 0, 0, 1, 1, 2, 0, 1, 0, 2, 1, 0, 1, 1, 0, 1, 1, 2, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 2, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 2, 1,
       1, 0, 0, 2, 1, 0, 1, 2, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 2, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 1, 2, 1, 2, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 2, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 2, 2, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 2, 0, 1, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 2, 0, 1, 1, 0, 1,
       1, 1, 0, 2, 0, 1, 1, 0, 1, 0, 1, 0, 2, 1, 1, 2, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 2, 1, 2, 1, 0, 2, 1, 0, 1, 0, 0,
       1, 2, 1, 0, 1, 1, 0, 0, 1, 1, 1, 2, 0, 2, 1, 1, 1, 1, 1, 1, 1, 0,
       2, 1, 1, 1, 0, 0, 1, 1, 1])

In [66]:
aapl_clusters = aapl_unsup.copy()
aapl_clusters['cluster'] = clusters
aapl_clusters.hvplot.scatter( x='high', y='low', by='cluster' )

:NdOverlay   [cluster]
   :Scatter   [high]   (low)

In [76]:
aapl_last15 = aapl_unsup.iloc[-15:,:]
last15_clusters = km.predict(aapl_last15)
last15_clustered = aapl_last15.copy()
last15_clustered['cluster']=last15_clusters
last15_clustered.hvplot.scatter(x='volume', y='vwap', by='cluster')

:NdOverlay   [cluster]
   :Scatter   [volume]   (vwap)

In [97]:
# 2=buy, 0=sell
last15_clustered

,open,high,low,close,volume,trade_count,vwap,cluster
timestamp,,,,,,,,
2023-08-09 04:00:00+00:00,0.006567,0.003661,-0.003210,-0.008954,-0.109137,-0.060411,-0.002776,1
2023-08-10 04:00:00+00:00,-0.007685,-0.000995,0.003333,-0.001235,-0.096338,-0.021969,0.001717,1
2023-08-11 04:00:00+00:00,-0.012035,-0.011784,-0.005912,-0.001011,-0.034335,-0.106115,-0.005583,1
2023-08-14 04:00:00+00:00,0.003666,0.005990,0.004276,0.009393,-0.172031,-0.029538,0.006476,1
2023-08-15 04:00:00+00:00,0.005113,-0.001169,-0.001438,-0.011200,-0.000496,-0.097264,-0.005361,1
2023-08-16 04:00:00+00:00,-0.009783,-0.005237,-0.003106,-0.004959,0.104464,0.031607,-0.002987,0
2023-08-17 04:00:00+00:00,0.000056,-0.005795,-0.017110,-0.014555,0.369154,0.275957,-0.016181,2
2023-08-18 04:00:00+00:00,-0.027323,-0.013551,-0.008762,0.002816,-0.080402,-0.123723,-0.003996,1
2023-08-21 04:00:00+00:00,0.016077,0.005882,0.010322,0.007737,-0.234871,-0.048964,0.007677,1


## scaler method

In [104]:
aapl_sc = aapl_df.drop('symbol',axis=1)
aapl_scaled = pd.DataFrame(StandardScaler().fit_transform(aapl_sc),
                           index=aapl_sc.index,
                           columns=aapl_sc.columns)
aapl_scaled

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2022-08-29 04:00:00+00:00,0.077966,0.078643,0.091992,0.083108,0.162422,0.160426,0.093752
2022-08-30 04:00:00+00:00,0.131365,0.059958,-0.020214,-0.050855,0.353506,0.185474,-0.027349
2022-08-31 04:00:00+00:00,0.032427,-0.048856,-0.051205,-0.142514,0.772499,-0.053710,-0.075845
2022-09-01 04:00:00+00:00,-0.166260,-0.167562,-0.183181,-0.102379,0.200393,0.248386,-0.177311
2022-09-02 04:00:00+00:00,0.002340,-0.060836,-0.167419,-0.218986,0.319225,0.195246,-0.116210
...,...,...,...,...,...,...,...
2023-08-23 04:00:00+00:00,1.019903,1.103583,1.080745,1.153727,-0.692412,-0.163518,1.129100
2023-08-24 04:00:00+00:00,1.136677,1.079072,0.957050,0.896648,-0.602182,-0.029315,0.969631
2023-08-25 04:00:00+00:00,0.958101,0.971687,0.946898,1.017594,-0.772650,-0.424593,0.980857


In [105]:
km_sc = KMeans(n_clusters=3, n_init=99)
km_sc.fit(aapl_scaled)
aapl_sc_pred = km_sc.predict(aapl_scaled)
aapl_scd = aapl_sc.copy()
aapl_scd['cluster'] = aapl_sc_pred
aapl_scd

,open,high,low,close,volume,trade_count,vwap,cluster
timestamp,,,,,,,,
2022-08-29 04:00:00+00:00,161.145,162.900,159.820,161.38,73313953,640593,161.529078,1
2022-08-30 04:00:00+00:00,162.130,162.560,157.720,158.91,77896533,644584,159.292663,1
2022-08-31 04:00:00+00:00,160.305,160.580,157.140,157.22,87944856,606474,158.397062,1
2022-09-01 04:00:00+00:00,156.640,158.420,154.670,157.96,74224560,654608,156.523257,1
2022-09-02 04:00:00+00:00,159.750,160.362,154.965,155.81,77074411,646141,157.651625,1
...,...,...,...,...,...,...,...,...
2023-08-23 04:00:00+00:00,178.520,181.550,178.325,181.12,52813270,588978,180.649220,0
2023-08-24 04:00:00+00:00,180.674,181.104,176.010,176.38,54977169,610361,177.704249,0
2023-08-25 04:00:00+00:00,177.380,179.150,175.820,178.61,50888986,547380,177.911565,0


In [107]:
aapl_scd.hvplot.scatter(x='trade_count', y='close', by='cluster')

:NdOverlay   [cluster]
   :Scatter   [trade_count]   (close)

In [108]:
aapl_last15 = aapl_sc.iloc[-15:,:]
last15_clusters = km_sc.predict(aapl_last15)
last15_clustered = aapl_last15.copy()
last15_clustered['cluster']=last15_clusters
last15_clustered.hvplot.scatter(x='volume', y='vwap', by='cluster')

:NdOverlay   [cluster]
   :Scatter   [volume]   (vwap)

In [99]:
last15_clustered

,open,high,low,close,volume,trade_count,vwap,cluster
timestamp,,,,,,,,
2023-08-09 04:00:00+00:00,180.870,180.9300,177.010,178.19,60514820,667636,178.488650,2
2023-08-10 04:00:00+00:00,179.480,180.7500,177.600,177.97,54684936,652969,178.795121,2
2023-08-11 04:00:00+00:00,177.320,178.6200,176.550,177.79,52807349,583679,177.796978,2
2023-08-14 04:00:00+00:00,177.970,179.6900,177.305,179.46,43722839,566438,178.948368,2
2023-08-15 04:00:00+00:00,178.880,179.4800,177.050,177.45,43701172,511344,177.988964,2
2023-08-16 04:00:00+00:00,177.130,178.5400,176.500,176.57,48266387,527506,177.457238,2
2023-08-17 04:00:00+00:00,177.140,177.5054,173.480,174.00,66084135,673075,174.585876,2
2023-08-18 04:00:00+00:00,172.300,175.1000,171.960,174.49,60770859,589800,173.888267,2
2023-08-21 04:00:00+00:00,175.070,176.1300,173.735,175.84,46497547,560921,175.223265,2


### try PCA

In [113]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
aapl_pca = pd.DataFrame(pca.fit_transform(aapl_scaled), index=aapl_scaled.index, columns=['PCA1','PCA2'])
#aapl_pca.set_index = aapl_scaled.index
aapl_pca

,PCA1,PCA2
timestamp,,
2022-08-29 04:00:00+00:00,0.121751,0.267981
2022-08-30 04:00:00+00:00,-0.073093,0.368213
2022-08-31 04:00:00+00:00,-0.314217,0.402150
2022-09-01 04:00:00+00:00,-0.422825,0.216758
2022-09-02 04:00:00+00:00,-0.346062,0.279947
...,...,...
2023-08-23 04:00:00+00:00,2.550717,0.083711
2023-08-24 04:00:00+00:00,2.319099,0.190682
2023-08-25 04:00:00+00:00,2.339945,-0.230376


In [114]:
km_pca = KMeans(n_clusters=3, n_init=99)
km_pca.fit(aapl_pca)
aapl_pca_pred = km_pca.predict(aapl_pca)
aapl_pcad = aapl_pca.copy()
aapl_pcad['cluster'] = aapl_pca_pred
aapl_pcad

,PCA1,PCA2,cluster
timestamp,,,
2022-08-29 04:00:00+00:00,0.121751,0.267981,0
2022-08-30 04:00:00+00:00,-0.073093,0.368213,0
2022-08-31 04:00:00+00:00,-0.314217,0.402150,0
2022-09-01 04:00:00+00:00,-0.422825,0.216758,0
2022-09-02 04:00:00+00:00,-0.346062,0.279947,0
...,...,...,...
2023-08-23 04:00:00+00:00,2.550717,0.083711,1
2023-08-24 04:00:00+00:00,2.319099,0.190682,1
2023-08-25 04:00:00+00:00,2.339945,-0.230376,1


In [116]:
aapl_pcad.hvplot.scatter(x='PCA1', y='PCA2', by='cluster')

:NdOverlay   [cluster]
   :Scatter   [PCA1]   (PCA2)

In [121]:
aapl_last15 = aapl_pca.iloc[-15:,:]
last15_clusters = km_pca.predict(aapl_last15)
last15_clustered = aapl_last15.copy()
last15_clustered['cluster']=last15_clusters
last15_clustered.hvplot.scatter(x='PCA1', y='PCA2', by='cluster')

:NdOverlay   [cluster]
   :Scatter   [PCA1]   (PCA2)

## Conclusion: The only one can fit into our 15-day trading scheme is buy pct_change, using 2 as buy, 0 as sell